In [2]:
%%capture
from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime

from hts import (
    screening_vih,
    sdata, 
    base,
    hts,
    hts_1824
)


InteractiveShell.ast_node_interactivity = "all"
date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [3]:
screening_vih.case_id.count()
sdata.case_id.count()
base.id_patient.count()

66898

66898

12879

In [17]:
base.pivot_table(
    values='id_patient',
    index = 'ovc_age',
    columns = 'hts',
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name='Total'
)

hts_1824.id_patient.count()
hts.id_patient.count()


hts,no,yes,Total
ovc_age,,,
10-14,4928,151,5079
15-17,3115,323,3438
18-24,3467,895,4362
Total,11510,1369,12879


895

1369

In [27]:
HTS = hts[['id_patient','departement','commune','age_in_year','ovc_age','hts']]

In [36]:
screens = screening_vih[['id_patient','ovc_age','total_score']]
screens.rename(
    columns={
        'ovc_age': 'age_int'
    },
    inplace=True
)

f_screens = screens[screens.id_patient.isin(HTS.id_patient)]
f_screens

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_36681/139968117.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  screens.rename(


,id_patient,age_int,total_score
66,96862,10-14,32
75,129775,18-24,24
97,124125,15-17,34
110,79404,18-24,16
122,83412,18-24,24
...,...,...,...
66610,135056,15-17,32
66727,94150,18-24,0
66759,79861,10-14,26
66787,88161,15-17,4


In [38]:
FSCREENS = f_screens.merge(
    HTS,
    on='id_patient',
    how='left'
)
FSCREENS

,id_patient,age_int,total_score,departement,commune,age_in_year,ovc_age,hts
0,96862,10-14,32,Ouest,Port-au-Prince,14,10-14,yes
1,129775,18-24,24,Ouest,Port-au-Prince,18,18-24,yes
2,124125,15-17,34,Ouest,Kenscoff,17,15-17,yes
3,79404,18-24,16,Artibonite,Verrettes,22,18-24,yes
4,83412,18-24,24,Artibonite,Desdunes,19,18-24,yes
...,...,...,...,...,...,...,...,...
1364,135056,15-17,32,Ouest,Port-au-Prince,15,15-17,yes
1365,94150,18-24,0,Artibonite,Desdunes,20,18-24,yes
1366,79861,10-14,26,Artibonite,Verrettes,12,10-14,yes
1367,88161,15-17,4,Nord,Plaine-du-Nord,16,15-17,yes


In [45]:
FSCREENS.total_score.mean().round()

def hts_eligible(score):
    return 'yes' if score >= FSCREENS.total_score.mean().round() else 'no'


19.0

In [46]:
FSCREENS.loc[:,'hts_eligible'] = FSCREENS.loc[:,'total_score'].apply(
    lambda score: hts_eligible(score)
)
FSCREENS.hts_eligible.value_counts()

# Global

In [50]:
screens.total_score.mean().round()

def screens_eligible(score):
    return 'yes' if score >= screens.total_score.mean().round() else 'no'


12.0

In [53]:
screens.total_score.value_counts()

4     23003
0      7684
6      7512
26     5361
24     4909
32     4009
34     2585
2      2467
28     1792
30     1774
22     1463
18     1104
16      689
20      681
12      456
8       409
14      361
40      196
36      183
10      153
38      107
Name: total_score, dtype: int64

In [54]:
screens.loc[:,'hts_eligible'] = screens.loc[:,'total_score'].apply(
    lambda score: screens_eligible(score)
)
screens.hts_eligible.value_counts()

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_36681/3967926286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  screens.loc[:,'hts_eligible'] = screens.loc[:,'total_score'].apply(


no     41228
yes    25670
Name: hts_eligible, dtype: int64

# Hypotheses

In [63]:
from pandas import to_datetime

In [66]:
screening_vih.interview_date = to_datetime(screening_vih.interview_date)

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_36681/2715233519.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  screening_vih.interview_date = to_datetime(screening_vih.interview_date)


In [73]:
fy22 = screening_vih[
    (screening_vih.interview_date >="2021-10-01")&
    (screening_vih.interview_date <="2022-03-31")
]

fy22.case_id.count()

9895

In [74]:
fy22.total_score.value_counts()

4     4248
6     1252
26     853
24     818
32     667
34     368
22     303
28     301
30     281
18     174
14     106
16     103
0       97
12      91
20      85
36      38
8       37
10      30
40      21
38      13
2        9
Name: total_score, dtype: int64

In [76]:
fy22.total_score.mean().round()
def fy22_eligible(score):
    return 'yes' if score >= fy22.total_score.mean().round() else 'no'

14.0

In [77]:
fy22.loc[:,'hts_eligible'] = fy22.loc[:,'total_score'].apply(
    lambda score: fy22_eligible(score)
)
fy22.hts_eligible.value_counts()

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_36681/3551468202.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fy22.loc[:,'hts_eligible'] = fy22.loc[:,'total_score'].apply(


no     5764
yes    4131
Name: hts_eligible, dtype: int64

In [80]:
top20 = fy22[fy22.total_score>=20]

In [81]:
fy22[fy22.id_patient.isin(HTS.id_patient)]

,case_id,id_patient,patient_code,total_score,lastname,firstname,interview_date,dob,age,commune,age_range,ovc_age,hts_eligible
75,004997ef-357b-495a-ab49-f042d9583b64,129775,PAP/DRMS/027353401,24,François,Youventha,2021-10-19,2003-09-20,18,port-au-prince,15-19,18-24,yes
202,00b93357-7578-4feb-8cb5-4128afaa503d,131454,PAP/DRMS/192365601,4,Pierre,Maudeline,2021-11-27,1998-09-19,23,port-au-prince,20-24,18-24,no
324,013684fe-9e5c-436e-84c8-2bd12b374af6,128298,PAP/DRMS/031342801,16,Joseph,Shelda,2021-10-22,2006-02-16,16,tabarre,15-19,15-17,yes
531,020136c7-8471-4bd5-8012-5a06cdbf5d3e,132103,GON/DRMS/057369801,16,antoine,tatiana,2021-12-21,2000-02-14,22,la_chapelle,20-24,18-24,yes
973,03ae16f8-a58c-4a19-bb50-d7f567f13587,155008,GON/DRMS/062402701,32,Arisnat,Violène,2022-01-14,2006-01-21,16,verrettes,15-19,15-17,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65154,f978f72b-fb36-4497-81c5-7277ac57336f,132125,GON/DRMS/057369823,4,Joseph,Christela,2021-10-20,1999-01-07,23,la_chapelle,20-24,18-24,no
65925,fc62201a-2f66-479a-80bb-53df7ab1720e,130188,GON/DRMS/197356806,4,Maurice,Mackdala,2021-11-25,2003-01-14,19,montrouis,15-19,18-24,no
66089,fd1764df-50c7-4261-8208-18de29058c10,129999,PAP/DRMS/178355018,4,Oxeant,Ashley Sephora,2021-11-27,2007-05-03,14,port-au-prince,10-14,10-14,no
66328,fdfefa98-3015-4fd3-9913-62473d7fc0f1,154559,PAP/DRMS/234399116,4,Décembre,Wilnada,2022-01-15,2002-09-15,19,delmas,15-19,18-24,no


In [82]:
hts

,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,has_preventive_vbg,number_of_condoms_sensibilize,number_condoms_sensibilization_date_in_the_interval,...,socioeco_app,prep,contraceptive,ps_1014,ps_1519,ps_2024,primary_only,primary_and_OneSecondary_services,completed_one_service,has_started_one_service
244,75528,NaN,Nord,Cap-Haïtien,17,0,yes,yes,1,0,...,yes,no,no,no,primary,no,invalid,full_primary_leastOneSecondary,invalid,no
251,75813,NaN,Ouest,Kenscoff,20,0,yes,yes,1,0,...,no,no,no,no,primary,no,invalid,full_primary_leastOneSecondary,invalid,no
275,75987,NaN,Artibonite,Dessalines,17,0,yes,yes,1,0,...,yes,no,no,no,primary,no,invalid,full_primary_leastOneSecondary,invalid,no
279,75920,NaN,Ouest,Kenscoff,20,0,yes,yes,1,0,...,no,no,no,no,primary,no,invalid,full_primary_leastOneSecondary,invalid,no
311,76349,NaN,Artibonite,Verrettes,21,0,yes,yes,0,0,...,no,no,no,no,primary,no,invalid,full_primary_leastOneSecondary,invalid,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11154,155749,NaN,Nord,Cap-Haïtien,21,0,yes,yes,3,3,...,no,no,no,no,no,primary,invalid,full_primary_leastOneSecondary,invalid,no
11155,124659,NaN,Nord,Milot,13,0,yes,yes,2,2,...,no,no,no,no,no,no,invalid,invalid,primary_part_services,no
11156,124660,NaN,Nord,Milot,13,0,yes,yes,2,2,...,no,no,no,no,no,no,invalid,invalid,primary_part_services,no
11157,124661,NaN,Nord,Milot,13,0,yes,yes,2,2,...,no,no,no,no,no,no,invalid,invalid,primary_part_services,no
